In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add
import cv2
import matplotlib.pyplot as plt
# example of loading an image with the Keras API
from keras.preprocessing.image import load_img
from PIL import Image,ImageFile
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import tensorflow as tf

In [3]:
import PIL

In [4]:
descriptions = None
with open("file.txt",'r') as f:
    descriptions= f.read()
    
json_acceptable_string = descriptions.replace("'","\"")
descriptions = json.loads(json_acceptable_string)

In [5]:
print(type(descriptions))

<class 'dict'>


In [6]:
print(descriptions["667626_18933d713e"])

['girl is stretched out in shallow water', 'girl wearing red and multi colored bikini is laying on her back in shallow water', 'little girl in red swimsuit is laying on her back in shallow water', 'young girl is lying in the sand while ocean water is surrounding her', 'girl wearing bikini lying on her back in shallow pool of clear blue water']


In [7]:
vocab = set()
for key in descriptions.keys():
    for caption in descriptions[key]:
        vocab.update(caption.split())

In [8]:
print(len(vocab))

8426


In [9]:
total_words=[]
for key in descriptions.keys():
    for caption in descriptions[key]:
        caption=caption.split()
        for i in caption:
            total_words.append(i)

In [10]:
print(len(total_words))

373303


In [11]:
import collections

counter=collections.Counter(total_words)
freq_cnt = dict(counter)

In [12]:
for x,y in freq_cnt.items():
    print(x,y)
    

child 1545
in 18986
pink 739
dress 348
is 9345
climbing 507
up 1302
set 109
of 6723
stairs 109
an 1932
entry 1
way 53
girl 3328
going 149
into 1074
wooden 284
building 510
little 1768
playhouse 6
the 18419
to 3176
her 1178
cabin 4
black 3848
dog 8138
and 8862
spotted 38
are 3504
fighting 133
tri 14
colored 221
playing 2008
with 7765
each 430
other 773
on 10745
road 398
white 3959
brown 2578
spots 29
staring 57
at 2915
street 944
two 5642
dogs 2125
different 46
breeds 5
looking 743
pavement 48
moving 41
toward 146
covered 372
paint 62
sits 577
front 1386
painted 64
rainbow 22
hands 246
bowl 30
sitting 1368
large 1236
small 1278
grass 1622
plays 526
fingerpaints 3
canvas 6
it 401
there 304
pigtails 14
painting 43
young 2630
outside 791
man 7274
lays 56
bench 375
while 1968
his 2357
by 1249
him 403
which 51
also 20
tied 15
sleeping 60
next 749
shirtless 104
lies 43
park 508
laying 189
holding 1324
leash 131
ground 357
orange 745
hat 682
starring 8
something 346
wears 115
glasses 206
gauge

bars 45
hangs 58
monkey 27
jungle 31
gym 53
ladder 32
swinging 146
saver 1
wings 52
floaties 13
hit 70
tracks 28
railroad 18
track 291
jean 26
capris 2
barefoot 45
bearded 46
turquoise 10
peace 28
stops 14
symbol 3
headdress 11
shaved 15
crazy 2
hairdo 1
earrings 8
bound 9
misty 6
heather 1
surfboards 5
prepare 6
participating 9
wake 5
boards 11
attempt 17
give 10
simultaneously 1
decide 1
signal 5
armor 1
examining 8
tire 99
racers 16
drove 1
motorists 1
oddly 2
designed 2
racing 227
bucket 62
beneath 17
boogie 19
trampoline 118
telephone 9
power 12
lines 14
poles 52
electrical 4
wires 5
puppies 55
chase 40
heavily 11
foliaged 1
cold 35
but 26
morning 6
sets 17
trekking 3
hello 7
kitty 7
figure 17
yellowish 1
slightly 16
floats 18
peers 9
bulky 1
snoopy 2
huge 94
scene 47
skimpy 12
rain 51
screams 6
squinting 3
still 14
hillside 30
skating 84
instructor 5
skate 75
hold 88
go 76
museum 13
couches 2
lie 4
puffy 7
tired 3
silhouetted 13
stunning 2
retriever 37
transportation 9
business 1

polka 16
dot 13
grove 2
presentation 6
hardhat 15
length 3
upset 5
streaming 3
tears 2
crosswalk 21
ok 4
relax 5
converse 5
maroon 8
bend 25
squat 4
carry 20
laborador 1
waring 3
adorn 1
angle 3
stadium 42
sprints 10
floppy 11
licking 32
explores 1
medium 27
sized 27
rangler 1
cargo 4
khakis 1
lounge 13
resort 7
sunbathers 2
plaza 16
european 10
faded 4
sharing 8
serves 3
feeds 6
cream 45
exhibit 8
skips 8
domes 1
design 2
orbs 1
involving 4
swine 1
pet 15
piglet 2
plank 9
acrobatic 5
stunts 7
deflated 7
gotten 1
led 4
bicycler 9
wearubg 1
pit 23
converging 1
cycling 8
cycles 2
diry 1
cannonball 2
unison 4
terrior 1
soaking 7
ends 10
packaged 1
gifts 1
presents 5
checked 7
trays 2
product 5
dead 26
humans 2
master 4
checks 9
muzzle 36
rottweiler 6
cooling 3
kisses 21
goodbye 2
start 9
schoolchildren 2
drifting 2
fat 10
dingo 1
crag 2
blow 26
wands 2
arcade 15
whack 2
mole 1
em 1
whacking 1
aliens 1
corndogs 5
showering 3
home 28
facility 3
bathes 1
watering 5
wrapped 23
work 18
tool 6


attempted 1
opposing 43
keeps 5
defending 6
university 10
rival 7
confronted 1
opposition 1
advance 1
would 1
tackler 3
eluding 1
defenders 5
avoid 5
mascot 6
wisconsin 1
endzone 1
punting 2
onward 3
possession 5
avoiding 2
fumble 1
clutches 2
scrimmage 2
gain 2
receiver 1
referees 2
contest 6
collaborating 1
officials 3
lifted 10
teamates 1
athletes 7
receive 2
snap 5
center 20
compete 18
pro 6
layer 2
oppsing 1
defensive 3
punts 1
kicker 3
jersay 1
footballs 1
appear 15
national 1
anthem 1
staff 2
manager 1
nike 4
excercises 1
stretches 13
turf 2
americans 3
fan 12
weas 1
armbands 3
commuters 2
fro 1
lolly 3
graphic 2
shelves 3
blues 3
brothers 4
impersonator 1
expressions 7
concerned 5
runners 17
parallel 10
sunhat 1
cute 9
tupperware 1
chains 5
chained 4
girt 1
messenger 3
paneled 2
comfort 2
moonwalk 2
sundress 5
colander 3
clowns 10
mad 1
strainer 1
lighthouse 3
tower 3
youngster 2
gigantic 1
handbags 1
ollie 6
hooding 1
isolated 1
submerges 3
waterskis 4
speeding 7
inflatbale 1


nicely 2
stuntman 2
crumbling 3
fend 1
soocerball 1
battle 6
argues 1
striking 7
archways 1
arches 1
curtain 6
quarry 1
worried 2
keeling 1
lantern 1
dump 1
pour 2
poured 2
zombies 1
everybody 1
zombie 3
horro 1
bloodied 1
haloween 1
resembles 1
visits 1
amish 3
sect 1
social 1
keeping 3
challenging 7
guarded 2
united 4
states 4
windshield 1
ax 1
wielding 4
kneepads 1
stamds 1
pring 1
joust 1
jousting 2
horsemen 1
slicker 2
handling 2
buss 1
london 1
ferrett 1
personnel 3
stomachs 2
cemetery 1
stereo 1
wetsuits 4
proud 1
fetched 1
odeon 2
chops 3
preserves 1
stocky 1
stalks 2
piles 4
husk 1
camcorder 6
films 5
enjoyable 1
bleached 1
plugging 2
pac 1
obama 21
desolate 3
campaign 2
supporter 3
presidential 2
election 2
biden 1
candidate 1
mate 2
democrat 1
supporters 1
political 4
chris 1
gregoire 1
heritage 1
bone 10
average 1
everyday 1
cop 7
handgun 5
potties 3
mop 3
portable 4
toilets 4
bathrooms 2
barack 2
touts 1
riwal 1
raceway 4
celebratory 1
yell 2
streched 1
brides 4
gowns 4
he

dunk 3
doge 1
eggs 5
egg 1
speaker 3
spoke 1
osme 1
meetinghall 1
cements 1
bannister 1
cyclers 1
amazing 2
signature 1
document 2
signed 1
sponsor 1
motorcycling 1
submissive 1
longish 1
seal 5
fighters 3
litle 1
tabloid 1
slush 1
peolple 1
looms 1
telegraph 1
skislope 1
dupar 1
write 1
fellows 1
slates 1
backlegs 1
ot 1
domed 1
protester 2
anti 1
tax 2
dont 1
me 1
bro 1
wrecks 1
carreis 1
fleecy 1
pins 3
blays 1
cbs 2
mic 1
question 1
smilling 2
experiences 1
excites 1
dancefloor 2
upwards 4
einstein 4
albert 2
polar 1
glassess 1
avrovulcan 2
googles 2
sweating 1
mesa 1
fooling 1
lifesavers 1
name 3
lanyard 1
vase 1
restroom 1
penske 3
indy 1
pits 1
playes 1
stringless 1
gondola 6
operators 1
gondoliers 1
venice 1
tones 1
proudly 1
spikes 3
islamic 1
holy 1
divind 1
gallopsing 1
booklets 1
telescopes 1
bouquet 7
dj 6
djs 1
entertain 2
roped 1
bullfighter 2
skewers 1
stabs 2
bullfight 1
sock 1
struck 2
bases 3
coached 1
needs 1
acrobat 1
entangles 1
twisted 2
shortstop 1
science 1
arm

In [13]:
sorted_freq= sorted(freq_cnt.items(),reverse=True,key=lambda x:int(x[1]))


sorted_freq=[x for x in sorted_freq if x[1]>10]
total_words = [x[0] for x in sorted_freq]

In [14]:
print(len(total_words))
    

1845


# Image feature extraction

In [15]:

path="./Images/"
def preprocess_img(img):
    img=cv2.imread(img)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    x = np.asarray(img, dtype=float)
    x=x/224
    x= np.expand_dims(x,axis=0)
    return x


In [16]:
img=preprocess_img(path+"667626_18933d713e.jpg")
print(img.shape)

(1, 224, 224, 3)


In [17]:
model=ResNet50(weights='imagenet',input_shape=(224,224,3))


# model.summary()

In [18]:

model_new = Model(model.input,model.layers[-2].output)

In [19]:
img=preprocess_img(path+"667626_18933d713e.jpg")
img_vector=model_new.predict(img)
print(img_vector.shape)

(1, 2048)


In [20]:
img_vector.reshape((-1,))
print(img_vector.shape)


(1, 2048)


In [21]:
def image_feature_extraction(img):
    img=preprocess_img(img)
    img_vector=model_new.predict(img)
    img_vector=img_vector.reshape((-1,))
    return img_vector
    
    

# train test split

In [22]:
key=[]
for i in descriptions:
    key.append(i)

In [23]:
print(len(key))

8091


In [24]:
train,test=train_test_split(key,test_size=0.2)

In [25]:
print(len(train))

6472


# image CNN

In [49]:
image_feature={}
for i in train:
   image_feature[i]=image_feature_extraction(path+i+".jpg")
    
    

KeyboardInterrupt: 

In [ ]:

with open("img_features.pkl","wb") as f:
     pickle.dump(image_feature,f)

In [50]:
with open('img_features.pkl', 'rb') as f:
    data = pickle.load(f)
    image_feature=data

In [51]:
image_feature_test={}
for i in test:
    image_feature_test[i]=image_feature_extraction(path+i+".jpg")

KeyboardInterrupt: 

In [ ]:
with open("img_test_features.pkl","wb") as f:
     pickle.dump(image_feature_test,f)

In [26]:
numtoword={}
wordtonum={}

for j,i in enumerate(total_words):
    numtoword[j+1]=i
    wordtonum[i]=j+1
    

In [27]:
print(len(wordtonum))
print(len(numtoword))
total_len=len(wordtonum)+3

1845
1845


In [28]:
wordtonum['startseq']=1846
numtoword[1846]='startseq'

In [29]:
wordtonum['endseq']=1847
numtoword[1847]='endseq'


In [30]:
maxlen=0
for key in descriptions.keys():
    for i in descriptions[key]:
        maxlen=max(maxlen,len(i.split()))
        

In [32]:
maxlen+=2


# data input

In [33]:
def inputdata(image_feature,descriptions,wordtonum,maxlen,batch_size):
    X1,X2, y = [],[],[]
    
    n =0
    while True:
        for key in image_feature.keys():
            n += 1
            
            photo = image_feature[key]
            for desc in descriptions[key] :
                seq=[]
                seq.append(wordtonum["startseq"])
                for word in desc.split():
                  if word in wordtonum:
                    seq.append(wordtonum[word])
                seq.append(wordtonum["endseq"])
                for i in range(0,len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]
                    
                    #0 denote padding word
                    xi = pad_sequences([xi],maxlen=maxlen,value=0,padding='post')[0]
                    yi = to_categorical([yi],num_classes=total_len)[0]
                    
                    X1.append(photo)
                    X2.append(xi)
                    y.append(yi)
                    
                if n==batch_size:
                    yield [[np.array(X1),np.array(X2)],np.array(y)]
                    X1,X2,y = [],[],[]
                    n = 0
         

# embedding 

In [35]:
f =open("glove.6B.50d.txt",encoding='utf8')

In [36]:

embedding_index = {}

for line in f:
    values = line.split()
    
    word = values[0]
    word_embedding = np.array(values[1:],dtype='float')
    embedding_index[word] = word_embedding

In [37]:
f.close()

In [38]:
embedding_index['and']

array([ 0.26818 ,  0.14346 , -0.27877 ,  0.016257,  0.11384 ,  0.69923 ,
       -0.51332 , -0.47368 , -0.33075 , -0.13834 ,  0.2702  ,  0.30938 ,
       -0.45012 , -0.4127  , -0.09932 ,  0.038085,  0.029749,  0.10076 ,
       -0.25058 , -0.51818 ,  0.34558 ,  0.44922 ,  0.48791 , -0.080866,
       -0.10121 , -1.3777  , -0.10866 , -0.23201 ,  0.012839, -0.46508 ,
        3.8463  ,  0.31362 ,  0.13643 , -0.52244 ,  0.3302  ,  0.33707 ,
       -0.35601 ,  0.32431 ,  0.12041 ,  0.3512  , -0.069043,  0.36885 ,
        0.25168 , -0.24517 ,  0.25381 ,  0.1367  , -0.31178 , -0.6321  ,
       -0.25028 , -0.38097 ])

In [39]:
def get_embedding_matrix():
    emb_dim = 50
    matrix = np.zeros((total_len,emb_dim))
    for word,idx in wordtonum.items():
        if embedding_index.get(word) is not None:
            embedding_vector = embedding_index.get(word)
            matrix[idx] = embedding_vector
            
    return matrix

In [40]:
embedding_matrix = get_embedding_matrix()
embedding_matrix.shape

(1848, 50)

In [41]:
embedding_matrix[7]

array([ 0.25616 ,  0.43694 , -0.11889 ,  0.20345 ,  0.41959 ,  0.85863 ,
       -0.60344 , -0.31835 , -0.6718  ,  0.003984, -0.075159,  0.11043 ,
       -0.73534 ,  0.27436 ,  0.054015, -0.23828 , -0.13767 ,  0.011573,
       -0.46623 , -0.55233 ,  0.083317,  0.55938 ,  0.51903 , -0.27065 ,
       -0.28211 , -1.3918  ,  0.17498 ,  0.26586 ,  0.061449, -0.273   ,
        3.9032  ,  0.38169 , -0.056009, -0.004425,  0.24033 ,  0.30675 ,
       -0.12638 ,  0.33436 ,  0.075485, -0.036218,  0.13691 ,  0.37762 ,
       -0.12159 , -0.13808 ,  0.19505 ,  0.22793 , -0.17304 , -0.07573 ,
       -0.25868 , -0.39339 ])

In [42]:
np.savetxt('embedding.txt', embedding_matrix, fmt='%f')

# image captioning model

In [76]:
input_captions = Input(shape=(maxlen,))
inp_cap1 = Embedding(input_dim=total_len,output_dim=50,mask_zero=True)(input_captions)
inp_cap2 = Dropout(0.5)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

In [77]:
decoder1 = add([inp_img2,inp_cap3])
decoder2 = Dense(256,activation='relu')(decoder1)
outputs = Dense(total_len,activation='softmax')(decoder2)

In [78]:
final_model=Model(inputs=[input_img_features,input_captions],outputs=outputs)

In [79]:
final_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 33)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 33, 50)       92400       input_6[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 2048)         0           input_5[0][0]                    
____________________________________________________________________________________________

In [80]:
final_model.layers[2].set_weights([embedding_matrix])
final_model.layers[2].trainable = False

In [81]:
final_model.compile(loss='categorical_crossentropy',optimizer="adam")

# training

In [79]:
epochs = 20
batch_size = 4
steps = len(train)//6
print(steps)

1078


In [80]:
for i in range(epochs):
        generator = inputdata(train,image_feature,image_feature_test,descriptions,wordtonum,maxlen,batch_size)
        final_model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
        final_model.save('model_'+str(i)+'.h5')

Epoch 1/1
1078/1078 [==============================] - 488s 452ms/step - loss: 4.9913
Epoch 1/1
 445/1078 [===========>..................] - ETA: 4:51 - loss: 4.4620

KeyboardInterrupt: 

# prediction

In [43]:
model = load_model('3_model_9.h5')

In [48]:

def predict_caption(photo):
    
    in_text = "startseq"
    for i in range(35):
        sequence = [wordtonum[w] for w in in_text.split() if w in wordtonum]
        sequence = pad_sequences([sequence],maxlen=35,padding='post')
        
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax() #WOrd with max prob always - Greedy Sampling
        word = numtoword[ypred]
        in_text += (' ' + word)
        
        if word == "endseq":
            break
    
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [52]:
img=preprocess_img(path+"755326139_ee344ece7b.jpg")

In [53]:
img_vector=model_new.predict(img)
print(img_vector.shape)

(1, 2048)


In [54]:
predict_caption(img_vector)

'two dogs are running through the grass'